In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as ss

In [47]:
df = pd.read_csv("adult_data/connections.csv")

In [48]:
# there are repeated connections, so we add them
df = df.groupby(["pre_root_id", "post_root_id"]).sum("syn_count").reset_index()

In [49]:
roots = set(df["pre_root_id"]).union(set(df["post_root_id"]))

root_id_to_index = {
    root_id: i for i, root_id in enumerate(roots)
}

In [50]:
# change pre and post root ids to indices
df["pre_root_id"] = df["pre_root_id"].map(root_id_to_index)
df["post_root_id"] = df["post_root_id"].map(root_id_to_index)

,pre_root_id,post_root_id,syn_count
count,2.701601e+06,2.701601e+06,2.701601e+06
mean,6.704676e+04,6.745899e+04,1.264836e+01
std,3.869784e+04,3.877633e+04,1.698593e+01
min,0.000000e+00,0.000000e+00,5.000000e+00
25%,3.359200e+04,3.378400e+04,6.000000e+00
50%,6.704000e+04,6.797400e+04,8.000000e+00
75%,1.006490e+05,1.011720e+05,1.300000e+01
max,1.341900e+05,1.341900e+05,2.405000e+03


In [53]:
# convert to sparse matrix
matrix = ss.coo_matrix((df["syn_count"], (df["pre_root_id"], df["post_root_id"])))

In [58]:
ss.save_npz("adult_data/good_synaptic_matrix.npz", matrix)

In [57]:
matrix

<134191x134191 sparse matrix of type '<class 'numpy.int64'>'
	with 2701601 stored elements in COOrdinate format>